<a href="https://colab.research.google.com/github/uvaizm/AIWorks/blob/master/Text_Extractive_Summary_REST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec  6 16:04:48 2018

@author: muvaiz
"""

# -*- coding: utf-8 -*-
"""
Created on Tue Nov  6 12:28:47 2018

@author: muvaiz
"""

import argparse

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict

from flask import Flask, request
from flask_restful import Resource, Api, reqparse
from sqlalchemy import create_engine
from json import dumps
from flask_jsonpify import jsonify

app = Flask(__name__)
api = Api(app)

parser = reqparse.RequestParser()
#parser.add_argument('task')
parser.add_argument('chat', help='Text to summarize')
parser.add_argument('length', type=int, help='Number of sentences to return')

'''def parse_arguments():
    """ Parse command line arguments """ 
    parser = argparse.ArgumentParser()
    parser.add_argument('filepath', help='File name of text to summarize')
    parser.add_argument('-l', '--length', default=4, help='Number of sentences to return')
    args = parser.parse_args()

    return args
'''

def read_file(text):
    """ Read the file at designated path and throw exception if unable to do so """ 
    try:
        formatted_text=[]
        #with open(path, 'r') as file:
        #    text=file.read()
        text_lines=text.split("\n")
        for line in text_lines:
            if line[-1] not in list(punctuation):
                line=line+"."
                
            formatted_text.append(line)
        text="\n".join(formatted_text)
        print("Conversation : ")
        print(text)
        return text
    
    except IOError as e:
        print("Fatal Error: File ({}) could not be locaeted or is not readable.".format(path))

def sanitize_input(data):
    """ 
    Currently just a whitespace remover. More thought will have to be given with how 
    to handle sanitzation and encoding in a way that most text files can be successfully
    parsed
    """
    replace = {
        ord('\f') : ' ',
        ord('\t') : ' ',
        ord('\n') : ' ',
        ord('\r') : None
    }

    return data.translate(replace)

def tokenize_content(content):
    stop_words=list(set(stopwords.words('english')))+list(punctuation)
    words=word_tokenize(content.lower())
    
    return([sent_tokenize(content),[word for word in words if word not in stop_words] ])

def score_tokens(filterd_words, sentence_tokens):
    """
    Builds a frequency map based on the filtered list of words and 
    uses this to produce a map of each sentence and its total score
    """
    word_freq = FreqDist(filterd_words)

    ranking = defaultdict(int)

    for i, sentence in enumerate(sentence_tokens):
        for word in word_tokenize(sentence.lower()):
            if word in word_freq:
                ranking[i] += word_freq[word]

    return ranking

def summarize(ranks, sentences, length):
    """
    Utilizes a ranking map produced by score_token to extract
    the highest ranking sentences in order after converting from
    array to string.  
    """
    if int(length) > len(sentences): 
        print("Error, more sentences requested than available. Use --l (--length) flag to adjust.")
        exit()

    indexes = nlargest(int(length), ranks, key=ranks.get)
    final_sentences = [sentences[j] for j in sorted(indexes)]
    return ' '.join(final_sentences)

class Summarizer(Resource):
    def post(self):
        """ Drive the process from argument to output """ 
        args = parser.parse_args()
        content = read_file(args['chat'])
        content = sanitize_input(content)
    
        sentence_tokens, word_tokens = tokenize_content(content)  
        sentence_ranks = score_tokens(word_tokens, sentence_tokens)
        #print("\nSummary :")
        #return self.summarize(sentence_ranks, sentence_tokens, args.length)
        return(jsonify(summarize(sentence_ranks, sentence_tokens, args['length'])))
        
api.add_resource(Summarizer, '/summarizer')
        
if __name__ == '__main__':
    app.run(debug=True)


    